In [1]:
import numpy as np
import timeit
import random
import pickle, gzip

In [2]:
from IO import Reader, Writer, Data, Sentence, Token
from feature import FeatureMapping
from eisner import Eisner
from model import Model

# Main

In [5]:
train_path_en_5k = "Treebanks/english/train/wsj_train.only-projective.first-5k.conll06"
reader_en_5k = Reader(train_path_en_5k)
train_data_en_5k = Data(reader_en_5k.read_file())
train_data_en_5k.sentences

 ...]

In [6]:
mapping_en_5k = FeatureMapping(train_data_en_5k.sentences)
mapping_en_5k.create_map()

Start time: 8.652391416
1000 sentences took 17.914218459 to map
2000 sentences took 39.267353459 to map
3000 sentences took 63.96708420900001 to map
4000 sentences took 98.810717709 to map
5000 sentences took 138.117167792 to map


In [7]:
model_en_5k = Model(train_data_en_5k, mapping_en_5k)

In [9]:
model_en_5k.train(5)

Start time: 2428.020590875
Epoch: 1
Time taken for past 100 sentences: 4.576695833000031
Time taken for past 200 sentences: 10.457128000000012
Time taken for past 300 sentences: 14.588591916000041
Time taken for past 400 sentences: 20.312308291000136
Time taken for past 500 sentences: 25.479420582999865
Time taken for past 600 sentences: 30.615048749999914
Time taken for past 700 sentences: 34.84838958299997
Time taken for past 800 sentences: 39.749225083000056
Time taken for past 900 sentences: 43.836054416000025
Time taken for past 1000 sentences: 50.680409832999885
Time taken for past 1100 sentences: 56.22226483300028
Time taken for past 1200 sentences: 61.28591795800003
Time taken for past 1300 sentences: 65.947939958
Time taken for past 1400 sentences: 70.62136054100029
Time taken for past 1500 sentences: 75.61333858299986
Time taken for past 1600 sentences: 81.3069412079999
Time taken for past 1700 sentences: 86.98382745800018
Time taken for past 1800 sentences: 92.549225708
Time

Time taken for past 4700 sentences: 195.5867106249998
Time taken for past 4800 sentences: 199.74595495799986
Time taken for past 4900 sentences: 203.58571574999996
Time taken for past 5000 sentences: 208.16937362499993
UAS score on 127143 tokens over 5000 sentences: 0.9576146543655569
Time taken for epoch: 208.32797737500005
0.0

Start time: 3117.696211541
Epoch: 4
Time taken for past 100 sentences: 3.7694867089999207
Time taken for past 200 sentences: 7.533061999999973
Time taken for past 300 sentences: 11.862632999999732
Time taken for past 400 sentences: 15.57107304200008
Time taken for past 500 sentences: 19.963328374999946
Time taken for past 600 sentences: 23.53218545899972
Time taken for past 700 sentences: 28.335005541999635
Time taken for past 800 sentences: 31.897865999999794
Time taken for past 900 sentences: 36.47107262500003
Time taken for past 1000 sentences: 40.809026958999766
Time taken for past 1100 sentences: 45.056185124999956
Time taken for past 1200 sentences: 49.8

In [10]:
dev_gold_path_en = "Treebanks/english/dev/wsj_dev.conll06.gold"
reader_en_dev = Reader(dev_gold_path_en)
dev_gold_data = Data(reader_en_dev.read_file())
dev_gold_data.sentences

 ...]

In [11]:
model_en_5k.make_predictions(dev_gold_data)

In [15]:
dev_gold_data.sentences[0].tokens[5].x

'0'

In [13]:
dev_gold_data.evaluate()

UAS score on 27678 tokens over 1083 sentences: 0.7957222342654816


0.7957222342654816

In [16]:
for dev_sentence in dev_gold_data.sentences:
    for token in dev_sentence.tokens:
        token.head = token.x
        token.x = "_"

In [18]:
dev_gold_data.sentences[0].tokens[5].head

'0'

In [22]:
dev_gold_path_en.split('/')[-1]

'wsj_dev.conll06.gold'

In [19]:
dev_en_writer = Writer(dev_gold_path_en, dev_gold_data.sentences)
dev_en_writer.write_file()

In [23]:
test_gold_path_en = "Treebanks/english/test/wsj_test.conll06.blind"
reader_en_test = Reader(test_gold_path_en)
test_gold_data = Data(reader_en_test.read_file())
test_gold_data.sentences

 ...]

In [24]:
model_en_5k.make_predictions(test_gold_data)

In [25]:
test_gold_data.sentences[0].tokens[1].x

'2'

In [26]:
for gold_sentence in test_gold_data.sentences:
    for token in gold_sentence.tokens:
        token.head = token.x
        token.x = "_"

In [28]:
test_gold_data.sentences[0].tokens[1].x

'_'

In [29]:
test_gold_writer = Writer(test_gold_path_en.split('/')[-1], test_gold_data.sentences)
test_gold_writer.write_file()

# German

In [30]:
train_path_de_5k = "Treebanks/german/train/tiger-2.2.train.only-projective.first-5k.conll06"
reader_de_train_5k = Reader(train_path_de_5k)
train_data_de_5k = Data(reader_de_train_5k.read_file())
train_data_de_5k.sentences

 ...]

In [31]:
mapping_de_5k = FeatureMapping(train_data_de_5k.sentences)
mapping_de_5k.create_map()

Start time: 4336.895447333
1000 sentences took 7.0917087079997145 to map
2000 sentences took 14.227211832999274 to map
3000 sentences took 21.759672874999524 to map
4000 sentences took 29.093026957999427 to map
5000 sentences took 37.45333033299994 to map


In [34]:
model_de_5k = Model(train_data_de_5k, mapping_de_5k)

In [35]:
model_de_5k.train(5)

Start time: 5143.440478875
Epoch: 1
Time taken for past 100 sentences: 1.7318960829998105
Time taken for past 200 sentences: 3.256585083000573
Time taken for past 300 sentences: 4.742964582999775
Time taken for past 400 sentences: 6.8191644160006035
Time taken for past 500 sentences: 8.058118208000451
Time taken for past 600 sentences: 9.345886707999853
Time taken for past 700 sentences: 10.842702083000404
Time taken for past 800 sentences: 11.858571583000412
Time taken for past 900 sentences: 13.186151165999945
Time taken for past 1000 sentences: 14.410520750000614
Time taken for past 1100 sentences: 15.732959291000043
Time taken for past 1200 sentences: 17.175592583000252
Time taken for past 1300 sentences: 18.290665416000593
Time taken for past 1400 sentences: 19.365257500000553
Time taken for past 1500 sentences: 20.6950059159999
Time taken for past 1600 sentences: 21.950873165999838
Time taken for past 1700 sentences: 23.19059420800022
Time taken for past 1800 sentences: 24.886943

Time taken for past 4700 sentences: 62.32188633300029
Time taken for past 4800 sentences: 63.35497241600024
Time taken for past 4900 sentences: 64.71199845800038
Time taken for past 5000 sentences: 65.80734604099962
UAS score on 79632 tokens over 5000 sentences: 0.9164531846493872
Time taken for epoch: 65.8891947080001
0.0

Start time: 5351.909699625
Epoch: 4
Time taken for past 100 sentences: 1.2629696249996414
Time taken for past 200 sentences: 2.5579237909996664
Time taken for past 300 sentences: 3.5926619159999973
Time taken for past 400 sentences: 4.889801041000283
Time taken for past 500 sentences: 6.4902797499999
Time taken for past 600 sentences: 7.8322649159999855
Time taken for past 700 sentences: 8.995308749999822
Time taken for past 800 sentences: 10.176428124999802
Time taken for past 900 sentences: 11.236458624999614
Time taken for past 1000 sentences: 12.541302290999738
Time taken for past 1100 sentences: 13.97939579099966
Time taken for past 1200 sentences: 15.285963958

In [36]:
# test on dev data
dev_de_path = "Treebanks/german/dev/tiger-2.2.dev.conll06.gold"
reader_de_dev = Reader(dev_de_path)
dev_data_de = Data(reader_de_dev.read_file())
len(dev_data_de.sentences)

5000

In [37]:
model_de_5k.make_predictions(dev_data_de)

In [38]:
dev_data_de.evaluate()

UAS score on 81704 tokens over 5000 sentences: 0.7838783902868892


0.7838783902868892

In [39]:
for dev_sentence in dev_data_de.sentences:
    for token in dev_sentence.tokens:
        token.head = token.x
        token.x = "_"

In [41]:
dev_de_path.split('/')[-1].rsplit('.', 1)[0]

'tiger-2.2.dev.conll06'

In [44]:
dev_writer_de = Writer(dev_de_path, dev_data_de.sentences)
dev_writer_de.write_file()

In [45]:
test_path_de = "Treebanks/german/test/tiger-2.2.test.conll06.blind"
reader_de_test = Reader(test_path_de)
test_data_de = Data(reader_de_test.read_file())
len(test_data_de.sentences)

5000

In [46]:
model_de_5k.make_predictions(test_data_de)

In [47]:
test_data_de.sentences[0].tokens[1].x

'2'

In [48]:
for gold_sentence in test_data_de.sentences:
    for token in gold_sentence.tokens:
        token.head = token.x
        token.x = "_"

In [49]:
test_data_de.sentences[0].tokens[1].head

'2'

In [50]:
test_writer_de = Writer(test_path_de.split('/')[-1], test_data_de.sentences)
test_writer_de.write_file()

In [54]:
import pickle

stream = gzip.open('parser', 'wb')
pickle.dump(model_de_5k,stream,-1)
stream.close()

In [55]:
stream = gzip.open('parser', 'rb')
dep_parser = pickle.load(stream)
stream.close()

In [56]:
dep_parser.make_predictions(test_data_de)

In [57]:
test_data_de.sentences[0].tokens[2].x

'6'

In [ ]:
from difflib import Differ
 
with open(train_path_en) as file_1, open(train_path_en.split('/')[-1]+'.pred') as file_2:
    differ = Differ()
 
    for line in differ.compare(file_1.readlines(), file_2.readlines()):
        print(line)